## MLBootcamp21 Week 4 Assignment: Gridsearch and CrossValidation
- This week we will focus on improving our existing models using these two techniques
- They are good approaches to improving your model's performance apart from feature engineering

## Q1. Use GridSearchCV and also explore other GridSearch techniques provided by scikit-learn to tune your hyperparameters and obtain the best model

- Try out Decision Trees, Random Forest, GradientBoost, AdaBoost and also XGBoost 
- Run the model on the features you have generated until now without tuning the parameters first to check the result on basic parameters
- Then apply GridSearchCV or other GridSearch techniques provided by scikit-learn to tune the hyperparameters and get results on the best model

## Keep the "random_state" number as 42 or anynumber of your choice and report that number for me to be able to reproduce the same results

- Report your performance on the test set after making the submission on kaggle. 

- ****Do not use some random existing notebook on Kaggle to get the best results as you will not learn anything that way and we will be able to easily know if that has been done. Do whatever you can****

In [236]:
# Write your code from this cell
# It need not be in a single cell

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from collections import Counter
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
!pip install category_encoders

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

from sklearn.model_selection import GridSearchCV

from sklearn import tree
import matplotlib.pyplot as plt
from sklearn import datasets

from xgboost import XGBClassifier

import category_encoders as ce
from sklearn.metrics import accuracy_score

In [237]:
df_train = pd.read_csv("./train.csv")
df_train.head()

df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [238]:

test = df_train[['Cabin', 'Survived']]
test['Cabin'] = test['Cabin'].fillna('N/A')
print(test[["Survived", "Cabin"]].value_counts())

print(481/687) #70% of people with no cabin name died

Survived  Cabin  
0         N/A        481
1         N/A        206
          B96 B98      4
          F33          3
          E101         3
                    ... 
          C101         1
          C103         1
          C104         1
          C106         1
0         A10          1
Length: 165, dtype: int64
0.7001455604075691


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [239]:
#fill in the missing values
df_train['Embarked'] = df_train['Embarked'].fillna('S')
df_train['Age'] = df_train.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))
df_train['Cabin'] = df_train['Cabin'].fillna('N') 
df_test.fillna(method="ffill",inplace=True)

print(df_train)

     PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0              1         0       3  ...   7.2500     N         S
1              2         1       1  ...  71.2833   C85         C
2              3         1       3  ...   7.9250     N         S
3              4         1       1  ...  53.1000  C123         S
4              5         0       3  ...   8.0500     N         S
..           ...       ...     ...  ...      ...   ...       ...
886          887         0       2  ...  13.0000     N         S
887          888         1       1  ...  30.0000   B42         S
888          889         0       3  ...  23.4500     N         S
889          890         1       1  ...  30.0000  C148         C
890          891         0       3  ...   7.7500     N         Q

[891 rows x 12 columns]


In [ ]:
df_train['Cabin'] = df_train['Cabin'].apply(lambda x: x[0])
print(df_train['Cabin'].value_counts())
df_train[['Cabin', 'Survived']].value_counts()

In [241]:

df_train['Titles'] = df_train['Name'].apply(lambda x:re.findall('[A-Za-z]+\.', x)[0])
df_train['Fam_Size'] = df_train['SibSp'] + df_train['Parch'] + 1
family_map = {1: 'Sin', 2: 'Min', 3: 'Min', 4: 'Med', 5: 'Med', 6: 'Med', 7: 'Big', 8: 'Big', 11: 'Big'}
df_train['Group_size'] = df_train['Fam_Size'].map(family_map)
df_train['Married'] = df_train['Titles'].apply(lambda x: 1 if x == "Mrs." else 0)
ce_OHE = ce.OneHotEncoder(cols=['Group_size'])
df_train = ce_OHE.fit_transform(df_train)

#normalize data 
df_train[['female','male']] = pd.get_dummies(df_train['Sex'])
df_train['Age_Normalized'] = MinMaxScaler().fit_transform(np.array(df_train['Age']).reshape(-1, 1))
df_train['Fare_Normalized'] = MinMaxScaler().fit_transform(np.array(df_train['Fare']).reshape(-1, 1))
df_train['Pclass_Normalized'] = MinMaxScaler().fit_transform(np.array(df_train['Pclass']).reshape(-1, 1))

# df_train[["C","Q","S"]] = pd.get_dummies(df_train["Embarked"])

df_train['Cabin'] = df_train['Cabin'].replace(['T', 'A', 'B', 'C'], 'high_class')
df_train['Cabin'] = df_train['Cabin'].replace(['D', 'E'], 'med_class')
df_train['Cabin'] = df_train['Cabin'].replace(['F', 'G'], 'low_class')

df_train[["N", "h_class", "m_class", "l_class"]] = pd.get_dummies(df_train['Cabin'])

drop_feat = ['PassengerId', 'Titles', 'Sex', 'Cabin', 'Embarked', 'Name', 'Fam_Size', 'Fare', 'SibSp', 'Parch', 'Age', 'Ticket', 'Pclass']
df_train.drop(drop_feat, axis=1, inplace=True)



/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [242]:
x_train, x_test, y_train, y_test = train_test_split(df_train.loc[:, 'Group_size_1':] , df_train.Survived, test_size = 0.2, random_state = 40)



In [243]:

model = LogisticRegression()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
print(classification_report(y_test, y_predict)) 

              precision    recall  f1-score   support

           0       0.85      0.86      0.86       103
           1       0.81      0.79      0.80        76

    accuracy                           0.83       179
   macro avg       0.83      0.83      0.83       179
weighted avg       0.83      0.83      0.83       179



In [244]:
dt_model = DecisionTreeClassifier()
dt_model.fit(x_train, y_train)
dt_model_predictions = dt_model.predict(x_test)


print(classification_report(y_test,dt_model_predictions))

              precision    recall  f1-score   support

           0       0.79      0.83      0.81       103
           1       0.75      0.71      0.73        76

    accuracy                           0.78       179
   macro avg       0.77      0.77      0.77       179
weighted avg       0.78      0.78      0.78       179



In [245]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train,y_train)

rf_model_predictions = rf_model.predict(x_test)
print(classification_report(y_test,rf_model_predictions))

              precision    recall  f1-score   support

           0       0.80      0.87      0.83       103
           1       0.80      0.70      0.75        76

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



In [246]:
xgb_model = XGBClassifier(learning_rate=0.3, max_depth= 7, min_child_weight=10, n_estimators= 150)
xgb_model.fit(x_train,y_train)

xgb_model_predictions = xgb_model.predict(x_test)
print(classification_report(y_test,xgb_model_predictions))



              precision    recall  f1-score   support

           0       0.87      0.87      0.87       103
           1       0.83      0.82      0.82        76

    accuracy                           0.85       179
   macro avg       0.85      0.84      0.85       179
weighted avg       0.85      0.85      0.85       179



In [170]:
xgb_parameter = {
    'max_depth': [1, 3, 5, 7],
    'learning_rate': [0.1, 0.3, 0.8],
    'n_estimators': [50, 100, 150, 200, 250],
    'min_child_weight': [0.5, 1, 2, 10, 50, 100]
}

In [171]:
xgb_gridSearch = GridSearchCV(estimator =  XGBClassifier(), param_grid = xgb_parameter )

In [172]:
xgb_gridSearch.fit(x_train, y_train)

KeyboardInterrupt: ignored

In [174]:
xgb_gridSearch.best_params_

AttributeError: ignored

In [247]:
gb_model = GradientBoostingClassifier(learning_rate=0.2, loss='deviance', max_depth= 3, n_estimators= 50)
gb_model.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.2, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [248]:

gb_model_predictions = gb_model.predict(x_test) 
print(classification_report(y_test, gb_model_predictions))



              precision    recall  f1-score   support

           0       0.84      0.92      0.88       103
           1       0.88      0.76      0.82        76

    accuracy                           0.85       179
   macro avg       0.86      0.84      0.85       179
weighted avg       0.86      0.85      0.85       179



In [12]:
#Grid Search CV
parameter_grid = {
    'loss':['deviance', 'exponential'],
    'learning_rate':[0.1, 0.2, 0.3, 0.5, 0.8, 1], 
    'n_estimators':[10, 20, 50, 100, 120, 500], 
    'max_depth': [1, 2, 3, 6, 12, 20, 50, 100]
}

In [13]:
gb_gridSearch = GridSearchCV(cv=5, estimator =  GradientBoostingClassifier(), param_grid = parameter_grid )

In [179]:
gb_gridSearch.fit(x_train, y_train)

KeyboardInterrupt: ignored

In [180]:
gb_gridSearch.best_params_

AttributeError: ignored

In [249]:
df_train.head()

,Survived,Group_size_1,Group_size_2,Group_size_3,Group_size_4,Married,female,male,Age_Normalized,Fare_Normalized,Pclass_Normalized,N,h_class,m_class,l_class
0,0,1,0,0,0,0,0,1,0.271174,0.014151,1.0,1,0,0,0
1,1,1,0,0,0,1,1,0,0.472229,0.139136,0.0,0,1,0,0
2,1,0,1,0,0,0,1,0,0.321438,0.015469,1.0,1,0,0,0
3,1,1,0,0,0,1,1,0,0.434531,0.103644,0.0,0,1,0,0
4,0,0,1,0,0,0,0,1,0.434531,0.015713,1.0,1,0,0,0


In [251]:
df_test = pd.read_csv("./test.csv")
df_test.head()


print(df_test.isnull().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [252]:
df_test['Embarked'] = df_test['Embarked'].fillna('S')
df_test['Age'] = df_test.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))
df_test['Cabin'] = df_test['Cabin'].fillna('N') 
df_test['Cabin'] = df_test['Cabin'].apply(lambda x: x[0])
df_test.fillna(method="ffill",inplace=True)

In [253]:
df_test['Fam_Size'] = df_test['SibSp'] + df_test['Parch'] + 1
family_map = {1: 'Sin', 2: 'Min', 3: 'Min', 4: 'Med', 5: 'Med', 6: 'Med', 7: 'Big', 8: 'Big', 11: 'Big'}
df_test['Group_size'] = df_test['Fam_Size'].map(family_map)

df_test['Titles'] = df_test['Name'].apply(lambda x:re.findall('[A-Za-z]+\.', x)[0])
df_test['Married'] = df_test['Titles'].apply(lambda x: 1 if x == "Mrs." else 0)

ce_OHE = ce.OneHotEncoder(cols=['Group_size'])


df_test = ce_OHE.fit_transform(df_test)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [254]:


df_test[['female','male']] = pd.get_dummies(df_test['Sex'])
df_test['Age_Normalized'] = MinMaxScaler().fit_transform(np.array(df_test['Age']).reshape(-1, 1))
df_test['Fare_Normalized'] = MinMaxScaler().fit_transform(np.array(df_test['Fare']).reshape(-1, 1))
df_test['Fare_Normalized'] = MinMaxScaler().fit_transform(np.array(df_test['Fare']).reshape(-1, 1))
df_test['Pclass_Normalized'] = MinMaxScaler().fit_transform(np.array(df_test['Pclass']).reshape(-1, 1))

# df_test[["C","Q","S"]] = pd.get_dummies(df_test["Embarked"])

df_test['Cabin'] = df_test['Cabin'].replace(['T', 'A', 'B', 'C'], 'high_class')
df_test['Cabin'] = df_test['Cabin'].replace(['D', 'E'], 'med_class')
df_test['Cabin'] = df_test['Cabin'].replace(['F', 'G'], 'low_class')

df_test[["N", "h_class", "m_class", "l_class"]] = pd.get_dummies(df_test['Cabin'])

In [255]:
drop_feat = ['Titles', 'Sex', 'Cabin', 'Embarked', 'Name', 'Fam_Size', 'Fare', 'SibSp', 'Parch', 'Age', 'Ticket', 'Pclass']
df_test.drop(drop_feat, axis=1, inplace=True)



In [256]:
print(df_train.shape)
print(df_test.shape)

print(df_train.columns)
print(df_test.columns)

(891, 15)
(418, 15)
Index(['Survived', 'Group_size_1', 'Group_size_2', 'Group_size_3',
       'Group_size_4', 'Married', 'female', 'male', 'Age_Normalized',
       'Fare_Normalized', 'Pclass_Normalized', 'N', 'h_class', 'm_class',
       'l_class'],
      dtype='object')
Index(['PassengerId', 'Group_size_1', 'Group_size_2', 'Group_size_3',
       'Group_size_4', 'Married', 'female', 'male', 'Age_Normalized',
       'Fare_Normalized', 'Pclass_Normalized', 'N', 'h_class', 'm_class',
       'l_class'],
      dtype='object')


In [257]:
submission_prediction = gb_model.predict(df_test.loc[:, 'Group_size_1':])

In [258]:
df_submission = df_test[['PassengerId']].copy()

In [259]:
df_submission['Survived'] = submission_prediction

In [260]:
df_submission.to_csv("submission_v1.csv", index = False)


#0.78468 PREDICTION   

## Q2. There might be times when you would only like to do K-Fold Cross Validation and not run the time consuming GridSearch everytime. That is what you will be doing in this question. 

- Read the documention for K-Fold crossvalidation provided by scikit-learn here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

- Try to understand and run the code provided by scikit-learn. Basically this function provided by scikit-learn will split your dataset into the "K" folds and will give you the indices of the "K" folds

- Then use the K-Fold crossvalidation technique to generate the "K" folds and report the accuracy obtained for each of the "K" folds

- You should get "K" different accuracy values and then finally take the average of all the "K" accuracies which would be your final model performance

- This exercise will help you really understand K-Fold CrossValidation

In [22]:
## Write your code from here
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVR

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

In [23]:
kf = KFold(n_splits=4, random_state=None, shuffle=False)

print(kf)

X = df_train.iloc[:, 1:]
y = df_train.iloc[:, 0]

print(X)

KFold(n_splits=4, random_state=None, shuffle=False)
     Group_size_1  Group_size_2  ...  Fare_Normalized  Pclass_Normalized
0               1             0  ...         0.014151                1.0
1               1             0  ...         0.139136                0.0
2               0             1  ...         0.015469                1.0
3               1             0  ...         0.103644                0.0
4               0             1  ...         0.015713                1.0
..            ...           ...  ...              ...                ...
886             0             1  ...         0.025374                0.5
887             0             1  ...         0.058556                0.0
888             0             0  ...         0.045771                1.0
889             0             1  ...         0.058556                0.0
890             0             1  ...         0.015127                1.0

[891 rows x 10 columns]


In [26]:
scores = []
svm_scores = []

kf = KFold(n_splits=6)

for train_index, test_index in kf.split(X):
    x_k_train, x_k_test, y_k_train, y_k_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
  
    xgb_model.fit(x_k_train, y_k_train)
    scores.append(xgb_model.score(x_k_test, y_k_test)) 
    


print(np.mean(scores))

0.8170536308120201


In [207]:
from sklearn import svm

accuracy = cross_val_score(model, x, y, scoring='accuracy', cv = 10)
# Find accuracy of kmodel
print(accuracy.mean())

0.6936953807740325


In [ ]:
## Thats it for this week. There are only two questions but they are a bit time consuming so enjoy! I hope you reach atleast 85-90% on Kaggle before tapping into Deep Learning

